<a href="https://colab.research.google.com/github/JonasHell/aml_monet/blob/main/notebooks/cINN_jonas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>